# Importing the data set

In [2]:
import os
import pandas as pd

test_path = ".\data\\test\\"
pos_path = ".\data\\train\\pos\\"
neg_path = ".\data\\train\\neg\\"

neg_review_list = []
pos_review_list = []
review_list = []
test_set = []

for file in os.listdir(neg_path):
    file_path = os.path.join(neg_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {}
    entry['class'] = 0
    entry['raw_txt'] = fh.read()
    neg_review_list.append(entry)
    fh.close()
    
for file in os.listdir(pos_path):
    file_path = os.path.join(pos_path, file)
    fh = open(file_path, 'r', encoding="utf8")
    entry = {}
    entry['class'] = 1
    entry['raw_txt'] = fh.read()
    pos_review_list.append(entry)
    fh.close()

review_list = pd.DataFrame(neg_review_list + pos_review_list)
print("done")

done


Preprocessing


Simple text preprocessing using stopwords


In [ ]:
import re
from nltk.corpus import stopwords
import numpy as np

stopWords = set(stopwords.words('english'))

def processing(df):
    #lowering and removing punctuation
    df['processed'] = df['raw_txt'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
        
    
    #numerical feature engineering
    #total length of sentence
    df['length'] = df['processed'].apply(lambda x: len(x))
    #get number of words
    df['words'] = df['processed'].apply(lambda x: len(x.split(' ')))
    df['words_not_stopword'] = df['processed'].apply(lambda x: len([t for t in x.split(' ') if t not in stopWords]))
    #get the average word length
    df['avg_word_length'] = df['processed'].apply(lambda x: np.mean([len(t) for t in x.split(' ') if t not in stopWords]) if len([len(t) for t in x.split(' ') if t not in stopWords]) > 0 else 0)
    #get the average word length
    df['commas'] = df['raw_txt'].apply(lambda x: x.count(','))

    return df


df = processing(review_list)
print(df.head())


In [4]:
from sklearn.model_selection import train_test_split
feature_set = [c for c in df.columns.values if c not in ['class']]
x_train, x_test, y_train, y_test = train_test_split(df[feature_set], df['class'], test_size=0.20, random_state=10)
print(x_test.head())
print(y_test.head())

                                                 raw_txt  \
18634  This is the best movie I have ever seen.<br />...   
1333   My interest in Dorothy Stratten caused me to p...   
20315  Whilst reading through the comments left for t...   
6357   'A Smile LIke Yours' is a pathetic comedy that...   
10496  Alien Hunter: 5 out of 10: Is it me or does ev...   

                                               processed  length  words  \
18634  this is the best movie i have ever seenbr br i...     702    141   
1333   my interest in dorothy stratten caused me to p...     452     78   
20315  whilst reading through the comments left for t...     858    165   
6357   a smile like yours is a pathetic comedy that a...    3277    570   
10496  alien hunter 5 out of 10 is it me or does ever...    2008    372   

       words_not_stopword  avg_word_length  commas  
18634                  73         5.150685       8  
1333                   41         6.609756       1  
20315                  85    

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator,TransformerMixin):
    def __init__(self, key):
        
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

text = Pipeline([
                ('selector', TextSelector(key='processed')),
                ('tfidf', TfidfVectorizer( stop_words='english'))
            ])


In [8]:
from sklearn.pipeline import FeatureUnion
features = FeatureUnion([('text', text),
                         
                         ])


In [ ]:
from sklearn.naive_bayes import BernoulliNB
from src.Naive_Bayes import NaiveBayes
pipeline = Pipeline([('features', features),
                     ('classifier', BernoulliNB())])
pipeline.fit(x_train, y_train)

prediction = pipeline.predict(x_test)
np.mean(prediction == y_test)
